In [1]:
from random import randrange, getrandbits

In [116]:
def is_prime(n, k=128):  # перевірка на простоту: імовірносний тест Мілера-Рабіна
    if n == 2 or n == 3: # <3 прості числа
        return True
    if n <= 1 or n % 2 == 0: #відкидаємо від'ємні та парні
        return False
    s = 0
    r = n - 1
    while r & 1 == 0: #зменшуємо вдвійчи r доти, доки воно не стане непарним числом
        s += 1
        r //= 2
    for _ in range(k): #k разів виконуємо перевірку Мілера-Рабіна
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1: 
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1: #умова заперечення простоти
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

In [117]:
def generate_prime_candidate(length):  # генеруємо кандидата на просте число із заданою довжиною
    p = getrandbits(length)
    p |= (1 << length - 1) | 1 #прибираємо нулі з початку
    return p

In [118]:
def generate_prime_number(length=1024):  # генеруємо просте число із заданою довжиною
    p = 4
    while not is_prime(p, 5):
        p = generate_prime_candidate(length)
    return p

In [119]:
generate_prime_number(100)

766066332529003612484219478881

In [120]:
def pow_mod(x, y, m):  #обчислення (x ^ y) mod m
    binary = bin(y)[2:] #y - в бінарний
    x0 = x % m
    if binary[-1] == "1":
        c = x
    else:
        c = 1
    for i in range(len(binary) - 1):
        x1 = x0 ** 2 % m
        if binary[-i - 2] == "1":
            c = c * x1 % m
        x0 = x1
    return c

In [121]:
pow_mod(5,2,20)

5

In [122]:
def convert_to_cipher(m_str):  #перетворення числового повідомлення на текстове
    m_int = 0
    for ch in m_str[::-1]:
        m_int = m_int * 1000 + ord(ch)
    return m_int

In [123]:
x = convert_to_cipher("Hello")
print(x)

111108108101072


In [124]:
def convert_to_string(m_int):  #перетворення текстового повідомлення на числове
    m_str = "" 
    copy = m_int
    while copy != 0:
        copy, val = divmod(copy, 1000)
        m_str += chr(val)
    return m_str

In [125]:
convert_to_string(x)

'Hello'

In [126]:
def ExtendedEuclid(x, y):  #розширений алгоритм Евкліда
    x0, x1, y0, y1 = 1, 0, 0, 1
    while y > 0:
        q, x, y = int(x // y), y, x % y
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return y0  #вихідний коефіціент

In [127]:
def ext_eucl_pos(F, e):  # розв'язання (d * e) mod F = 1
    d = ExtendedEuclid(F, e)
    if d < 0:
        return d + F
    return d

In [128]:
def gcd(x, y): #НСД(х,у)
    a, q0 = x, y
    r = 1
    while r != 0:
        b, r = divmod(a, q0)
        a, q0 = q0, r
    return a

In [129]:
gcd(10,15)

5

In [130]:
class PK:  # представлення PK
    def __init__(self, e, n, name=""):
        self.e = e
        self.n = n
        self.name = name

    def encrypt(self, message):         #кодування відкритим ключем
        return pow_mod(convert_to_cipher(message), self.e, self.n) 

    def verify(self, message, signature):  #верифікація підпису
        int_m = convert_to_cipher(message)
        (message)
        int_sig = pow_mod(signature, self.e, self.n)
        return int_m == int_sig

In [131]:
class RSA: #представлення RSA
    def __init__(self, length, name="", e=1): 
        self.__e = e
        self.__d = 1
        self.__pk = None
        self.__name = name
        self.__p = generate_prime_number(length)
        self.__q = generate_prime_number(length)
        self.__n = self.__p * self.__q
        self.__F = (self.__p - 1) * (self.__q - 1)
        if self.__e == 1:
            while self.__F % self.__e == 0:
                self.__e = generate_prime_number(length)
        self.__d = ext_eucl_pos(self.__F, self.__e)

    def get_e(self):  
        return self.__e

    def get_pk(self):  
        if self.__pk is None:
            self.__pk = PK(self.get_e(), self.__n, self.__name)
        return self.__pk

    def get_name(self):
        return self.__name

    def decrypt(self, c):       # розкодування зашифрованого посилання за допомогою закритого ключа
        m = pow_mod(c, self.__d, self.__n)
        return convert_to_string(m)

    def sign(self, m):  #підпис повідомлення
        return pow_mod(convert_to_cipher(m), self.__d, self.__n)
    
    def send(self, pk, message): #надіслати конкретній людині (pk) підписане повідомлення
        signature = self.sign(message)
        return pow_mod(signature, pk.e, pk.n)
    
    def receive(self, pk, encoded): #отримати підписание повідомлення від конкретної людини
        full_message = pow_mod(encoded, self.__d, self.__n)
        text = convert_to_string(pow_mod(full_message, pk.e, pk.n))
        return text

In [132]:
Alex = RSA(1000, "Alex")
pkAlex = Alex.get_pk()

In [133]:
Boutros = RSA(1000, "Boutros")
pkBoutros = Boutros.get_pk()

In [134]:
x = pkAlex.encrypt("Hi, Alex")

In [135]:
Alex.decrypt(x)

'Hi, Alex'

In [136]:
code = Boutros.send(pk=pkAlex, message="Hello")

In [137]:
Alex.receive(pk=pkBoutros, encoded=code)

'Hello'

In [156]:
AlexMessage = "Verifying message"

In [160]:
AlexMessageSignature = Alex.sign(AlexMessage)

In [161]:
pkAlex.verify(AlexMessage, AlexMessageSignature)

True

In [138]:
# китайська теорема про лишки
def chin(m, b):
    res = 0
    Mult_all = 1
    for mi in m:
        Mult_all = Mult_all * mi

    for i in range(len(b)):
        M = int(Mult_all // m[i])
        delta = ext_eucl_pos(m[i], M)
        res += M * delta * b[i] % Mult_all
    return res % Mult_all

In [139]:
class Group:                #група
    def __init__(self, *argv):  
        self.__list = []
        self.__list.extend(argv)

    def add(self, pk):   
        for pki in self.__list:
            if pki == pk:
                return True
            if gcd(pki.n, pk.n) != 1:
                return False
        self.__list.append(pk)
        return True

    def remove(self, pk):       
        if pk in self.__list:
            self.__list.remove(pk)

    def get(self):             
        return self.__list.copy()

    def sendToAll(self, message, sender):     #надіслати всім повідомлення
        codes = []
        modules = []
        for member in self.__list:
            codes.append(sender.send(member, message))   
            modules.append(member.n)                       
        return chin(modules, codes)     

In [140]:
Participant1 = RSA(name='Participant1', length=1000)
pkP1 = Participant1.get_pk()

In [143]:
Participant2 = RSA(name='Participant2', length=1000)
pkP2 = Participant2.get_pk()

In [144]:
Participant3 = RSA(name='Participant3', length=1000)
pkP3 = Participant3.get_pk()

In [145]:
AlexGroup = Group(pkP1,pkP2,pkP3)

In [146]:
AlexMsg = AlexGroup.sendToAll(sender=Alex, message='Hi guys!')

In [147]:
Participant1.receive(encoded=AlexMsg, pk=pkAlex)

'Hi guys!'

In [148]:
Participant2.receive(encoded=AlexMsg, pk=pkAlex)

'Hi guys!'

In [149]:
Participant3.receive(encoded=AlexMsg, pk=pkAlex)

'Hi guys!'

In [150]:
AlexGroup.remove(pkP2)

In [151]:
AlexGroup.get()

[<__main__.PK at 0x1f7ad175438>, <__main__.PK at 0x1f7ad175f98>]

In [152]:
AlexMsg2 = AlexGroup.sendToAll(sender=Alex, message='Hi guys again, but not P2!')

In [153]:
Participant1.receive(encoded=AlexMsg2, pk=pkAlex)

'Hi guys again, but not P2!'

In [154]:
Participant2.receive(encoded=AlexMsg2, pk=pkAlex)

'\x17/͉ȿ\x7f˫̃\x8c2Ǫ̈́ʶǤͫϠ̓ʓÜÔ\x00\nęʹ͵ɽģ-ǼȘξ¤\x03ΦǞˣɞʞ̦ˤɑłϦ\x8fŸ«͍_ʸƻ͢ʵ=\x96ƿʝϞˀϊÙé¨ɺ]ƌ˱ϗƐʭ³¼ǹʣǄǑă!̳Ǫ§\x80ː\x9bĔ̘Țȕåǫȓή˧Ƈͼκ͌¼ʈƽƔʓϜƧ̒ƤȎ\x18ǃƮ˸͘\u03a2̼Ïͺz\xadĶɗ˿o˞DEċ\x0f̭ŒΥLu˵éɜ̻¨ƟƋͱ͙đ\rjý!zǗω̶dţ͞ƖȅÙáȼʯaSų˰̐óʅȃȊåęĭ͏˅@ųĤŃǴyǴͷ˱{ˮ̝τŌ̊\x1bËĬƷȚÍ˻i^ͼ4ô͚Ñ\t'

In [155]:
Participant3.receive(encoded=AlexMsg2, pk=pkAlex)

'Hi guys again, but not P2!'